In [13]:
import pandas as pd
import geopandas
import fiona

In [23]:
import matplotlib as mpl

In [6]:
import os
import pydeck

In [58]:
start_point = (6.1402,46.208742)

In [59]:
delta = 0.002
bbox =  (start_point[0] - delta, start_point[1]-delta, start_point[0] + delta, start_point[1]+delta) 

In [75]:
filepath = './data/ge/bases.gpkg'
bases = geopandas.read_file(filepath, bbox=bbox)

filepath = './data/ge/facades.gpkg'
facade = geopandas.read_file(filepath, bbox=bbox)

filepath = './data/ge/toits.gpkg'
toits = geopandas.read_file(filepath, bbox=bbox)


bases.columns = [s.lower() for s in bases.columns]
facade.columns = [s.lower() for s in facade.columns]
toits.columns = [s.lower() for s in toits.columns]

In [76]:

INITIAL_VIEW_STATE = pydeck.ViewState(
  latitude=start_point[1],
  longitude=start_point[0],
  zoom=14,
  max_zoom=16,
  min_zoom=8,
  pitch=45,
  bearing=0
)

In [62]:
MAPBOX_API_KEY = os.environ["MAPBOX_API_KEY"]

KeyError: 'MAPBOX_API_KEY'

In [63]:
heat_demand = pd.read_csv('data/ge_heat_demand.csv')

In [82]:
toits = toits.merge(heat_demand, on='egid')

norm = mpl.colors.LogNorm(vmin=1, vmax=toits.q_heat_total.max())
cmap = mpl.cm.get_cmap('plasma')

cmap_val = cmap(norm(toits.q_heat_total)) 

toits['color'] = (cmap_val* 255).astype(int).tolist()

# toits['color'] = (cmap_val* 255).astype(int).tolist()

In [77]:
facade = facade.merge(heat_demand, on='egid')

norm = mpl.colors.LogNorm(vmin=1, vmax=facade.q_heat_total.max())
cmap = mpl.cm.get_cmap('plasma')

cmap_val = cmap(norm(facade.q_heat_total)) 

facade['color'] = (cmap_val* 255).astype(int).tolist()


In [89]:
vals = []

In [90]:
def _f(*args, **kwargs):
    global vals
    vals.append((args, kwargs))


In [108]:
# AWS Open Data Terrain Tiles
TERRAIN_IMAGE = "https://s3.amazonaws.com/elevation-tiles-prod/terrarium/{z}/{x}/{y}.png"

# Define how to parse elevation tiles
ELEVATION_DECODER = {"rScaler": 256, "gScaler": 1, "bScaler": 1 / 256, "offset": -32768}

SURFACE_IMAGE = f"https://api.mapbox.com/v4/mapbox.satellite/{{z}}/{{x}}/{{y}}@2x.png?access_token={MAPBOX_API_KEY}"


# lights = pydeck.LightSettings(
#         lights_position=[start_point[0], start_point[1], 8000, -3.807751, 54.104682, 8000], ambient_ratio=0.4, diffuse_ratio=0.6,
#     lights_strength=0.1
#     )


# lights = pydeck.LightSettings(
#     number_of_lights=1,
#         lights_position=[start_point[0]+1, start_point[1]+1, 80], ambient_ratio=0.4, diffuse_ratio=0.6,
#     lights_strength=0.1
#     )


terrain_layer = pydeck.Layer(
    "TerrainLayer", elevation_decoder=ELEVATION_DECODER, texture=SURFACE_IMAGE, elevation_data=TERRAIN_IMAGE,
    # light_settings=lights

)


bases_layer = pydeck.Layer('GeoJsonLayer', 
                           data=bases, get_fill_color=[255,100,255], pickable=True
                          )
facade_layer = pydeck.Layer('GeoJsonLayer', 
                            data=facade, 
                            get_fill_color='color', 
                            line_width_scale=0.1,
                            pickable=True,
                            # stroked=False,
                            filled=True,
                            wireframe=True,
                            # light_settings=lights

                           )
# facade_layer = pydeck.Layer('GeoJsonLayer', data=facade, get_fill_color='color', pickable=True)
toits_layer = pydeck.Layer('GeoJsonLayer', 
                           data=toits, 
                           get_fill_color='color', 
                           pickable=True,
                           stroked=False,
                           filled=True,
                           wireframe=True,

                          )

r.deck_widget.on_view_state_change(_f)


r = pydeck.Deck(
    layers=[
        terrain_layer,
        bases_layer, 
        facade_layer, 
        toits_layer
    ],
    initial_view_state=INITIAL_VIEW_STATE,
    map_style='satellite'
)

r

{"initialViewState": {"bearing": 0, "latitude": 46.208742, "longitude": 6.1402, "maxZoom": 16, "minZoom": 8, "pitch": 45, "zoom": 14}, "layers": [{"@@type": "TerrainLayer", "elevationData": "https://s3.amazonaws.com/elevation-tiles-prod/terrarium/{z}/{x}/{y}.png", "elevationDecoder": {"bScaler": 0.00390625, "gScaler": 1, "offset": -32768, "rScaler": 256}, "id": "9435f240-fb5f-44e1-8bce-9ab9c6fcd8fe", "texture": "https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}@2x.png?access_token=pk.eyJ1IjoibWFuZ2Vjb2V1ciIsImEiOiI3MTlkMmQ4NmNlOGM2ZTcyY2QyYjYyZDFiZGI2MjE4YiJ9.T4geVfZv5dDW0yWh_8meNg"}, {"@@type": "GeoJsonLayer", "data": [{"altitude_ref": 394.12, "date_leve": "2011/04/01 00:00:00", "egid": 1011245, "geometry": {"coordinates": [[[[6.138669388485449, 46.210135074599265, 394.119100000127], [6.138673396722409, 46.21013897862342, 394.119100000127], [6.138716104224495, 46.21018058440996, 394.119100000127], [6.138530061727625, 46.21027312732044, 394.119100000127], [6.138494190607582, 46.210237956503555, 394.119100000127], [6.138487081319795, 46.21023128529983, 394.119100000127], [6.138482973444541, 46.21022742960345, 394.119100000127], [6.138447508210519, 46.21019414738667, 394.119100000127], [6.138427122404665, 46.210175005618105, 394.119100000127], [6.138614125741623, 46.210081239583054, 394.119100000127], [6.138669388485449, 46.210135074599265, 394.119100000127]]]], "type": "MultiPolygon"}, "surface_sol": 239.45}, {"altitude_ref": 392.24, "date_leve": "2011/04/01 00:00:00", "egid": 295070335, "geometry": {"coordinates": [[[[6.138673912741797, 46.21043123221274, 392.236200000159], [6.138582438955253, 46.21048002982533, 392.236200000159], [6.13854692054035, 46.21044749916526, 392.236200000159], [6.138548356472407, 46.2104470631688, 392.236200000159], [6.138614046507565, 46.210425596025814, 392.236200000159], [6.1386568806368444, 46.2104116010101, 392.236200000159], [6.138673912741797, 46.21043123221274, 392.236200000159]]]], "type": "MultiPolygon"}, "surface_sol": 32.07}, {"altitude_ref": 392.2, "date_leve": "2019/06/08 00:00:00", "egid": 1011180, "geometry": {"coordinates": [[[[6.138507725756813, 46.20924320083122, 392.197900000261], [6.138510171340339, 46.209243949657534, 392.197900000261], [6.1384312185039205, 46.20940112340244, 392.197900000261], [6.138223287224083, 46.209414135174036, 392.197900000261], [6.138176180437447, 46.209416835619386, 392.197900000261], [6.138176942414003, 46.209415318537026, 392.197900000261], [6.138178422878888, 46.20941524113317, 392.197900000261], [6.138225420795205, 46.20931878153992, 392.197900000261], [6.138239073819491, 46.20929159478801, 392.197900000261], [6.1382894496817, 46.20919113361997, 392.197900000261], [6.138507725756813, 46.20924320083122, 392.197900000261]]]], "type": "MultiPolygon"}, "surface_sol": 405.74}, {"altitude_ref": 394.17, "date_leve": "2019/06/08 00:00:00", "egid": 1011173, "geometry": {"coordinates": [[[[6.137881595721921, 46.20966579914642, 394.165700000012], [6.138156326942405, 46.209547198146076, 394.165700000012], [6.138266779973858, 46.20966822629519, 394.165700000012], [6.138053401702967, 46.20976091940714, 394.165700000012], [6.137991412321184, 46.209787847672516, 394.165700000012], [6.137881595721921, 46.20966579914642, 394.165700000012]]]], "type": "MultiPolygon"}, "surface_sol": 399.35}, {"altitude_ref": 394.78, "date_leve": "2019/06/08 00:00:00", "egid": 295077271, "geometry": {"coordinates": [[[[6.1381769427130894, 46.20941531904097, 394.778799999855], [6.138176180736531, 46.20941683612333, 394.778799999855], [6.1381747478075965, 46.20942023660896, 394.778799999855], [6.138050717949889, 46.20942278004722, 394.778799999855], [6.138013474543852, 46.20944004214478, 394.778799999855], [6.137948474428332, 46.209368709047965, 394.778799999855], [6.137978658540622, 46.209347730874086, 394.778799999855], [6.138028501456358, 46.20931222357497, 394.778799999855], [6.138034922194851, 46.20929853061664, 394.778799999855], [6.138045418063674, 46.20927634842631, 394.7787